<a href="https://colab.research.google.com/github/schwartz-cnl/Computational-Neuroscience-Class/blob/main/Neural%20Box/neuralBox1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Neural Box 1

Odelia Schwartz, Xu Pan


Partly based on Cold Spring Harbor tutorial by Chichilnisky. This is a tutorial for understanding linear filter properties of neurons, and generating neural spikes
based on a linear filter followed by a simple nonlinearity. This is part one of two tutorials. We are going to first show how to generate neural model responses, which we will later probe and "guess"/figure out the properties.

The model neuron includes one linear and one nonlinear component. We follow this by Poisson spiking. We have made several versions of model neurons based on Chichilnisky's tutorial.

## Generate a stimulus

In [ ]:
# We want to first choose experimental stimuli that are random.
# At each frame, the intensity of the uniform screen changes:
# it is drawn randomly, here from a Gaussian distribution.

import numpy as np

numsamples = 25000
stimulus=(1/3*np.random.normal(size=numsamples))

In [ ]:
# Plot the stimulus

import matplotlib.pyplot as plt

plt.plot(stimulus[1:1000])
plt.xlabel('Time (msec)', fontsize=16)
plt.ylabel('Stimulus strength', fontsize=16)
plt.title('First 1 second of stimulus', fontsize=16)

## Neuron models

This cell has 5 functions getLinear1, getLinear2, getLinear3, getNonlinear1, getNonlinear2. You can think of them as "neurons" that take stimulus as input and give response as output.

***You don't have to look inside. Just run it.***

In [ ]:
def getLinear1(stimulus, kernelSize):
  # Compute the linear response using a single exponential
  # lowpass filter.  You could substitute other linear filters here if you
  # wanted to, but this one is pretty simple.
  tau = 10 # time constant
  linearResp = np.zeros(len(stimulus))
  
  for i in range(len(stimulus)-1):
    # Solve the differential equation
    linearResp[i+1] = linearResp[i] + (1/tau)*(stimulus[i]-linearResp[i]) 
  
  # get the impulse response function which is also the "filter"
  impulse = np.zeros(1000) # make a impulse stimulus
  impulse[0] = 1
  impulseResp = np.zeros(len(impulse))
  for i in range(len(impulse)-1):
    # Solve the differential equation
    impulseResp[i+1] = impulseResp[i] + (1/tau)*(impulse[i]-impulseResp[i])
  impulseResp = impulseResp[0:kernelSize]
  filter = np.flipud(impulseResp)

  return (linearResp, filter)

###############################################################################
def getLinear2(stimulus, kernelSize):
  # Compute the linear response using a single exponential
  # lowpass filter.  You could substitute other linear filters here if you
  # wanted to, but this one is pretty simple.
  tau = 5 # time constant
  linearResp = np.zeros(len(stimulus))
  
  for i in range(len(stimulus)-1):
    # Solve the differential equation
    linearResp[i+1] = linearResp[i] + (1/tau)*(stimulus[i]-linearResp[i]) 
  
  # get the impulse response function which is also the "filter"
  impulse = np.zeros(1000) # make a impulse stimulus
  impulse[0] = 1
  impulseResp = np.zeros(len(impulse))
  for i in range(len(impulse)-1):
    # Solve the differential equation
    impulseResp[i+1] = impulseResp[i] + (1/tau)*(impulse[i]-impulseResp[i])
  impulseResp = impulseResp[0:kernelSize]
  filter = np.flipud(impulseResp)

  linearResp = -linearResp
  filter = -filter

  return (linearResp, filter)

###############################################################################
def getLinear3(stimulus, kernelSize):
  # Compute the linear response using a 3-stage cascade of exponential
  # lowpass filters.  You could substitute other linear filters here if you
  # wanted to, but this one is pretty simple.
  tau = 3 # time constant
  linearResp = np.zeros((len(stimulus),3))
  
  for i in range(len(stimulus)-1):
    # Solve the differential equation
    linearResp[i+1,0] = linearResp[i,0] + (1/tau)*(stimulus[i]-linearResp[i,0])
    linearResp[i+1,1] = linearResp[i,1] + (1/tau)*(linearResp[i,0]-linearResp[i,1])
    linearResp[i+1,2] = linearResp[i,2] + (1/tau)*(linearResp[i,1]-linearResp[i,2])
  
  # Getting rid of the first- and second-order filtered signals, we only
  # want the third one.
  linearResp = linearResp[:,2]
  
  # get the impulse response function which is also the "filter"
  impulse = np.zeros(1000) # make a impulse stimulus
  impulse[0] = 1
  impulseResp = np.zeros((len(stimulus),3))
  for i in range(len(impulse)-1):
    # Solve the differential equation
    impulseResp[i+1,0] = impulseResp[i,0] + (1/tau)*(impulse[i]-impulseResp[i,0])
    impulseResp[i+1,1] = impulseResp[i,1] + (1/tau)*(impulseResp[i,0]-impulseResp[i,1])
    impulseResp[i+1,2] = impulseResp[i,2] + (1/tau)*(impulseResp[i,1]-impulseResp[i,2])

  # Getting rid of the first- and second-order filtered signals, we only
  # want the third one.
  impulseResp = impulseResp[:,2]

  impulseResp = impulseResp[0:kernelSize]
  filter = np.flipud(impulseResp)

  return (linearResp, filter)

###############################################################################
def getNonlinear1(linearResp):
  nonlinearResp = np.zeros(len(linearResp))
  theind = np.where(linearResp>0)
  nonlinearResp[theind] = linearResp[theind]**2
  return nonlinearResp

###############################################################################
def getNonlinear2(linearResp):
  return linearResp**2

## Linear model

In [ ]:
# We're now going to simulate a model neuron
# For purposes of this demo, we constructed the model
# neurons and so know their filters and nonlinearity
# (in an experiment with real neurons, we would be handed 
# the spike trains and would not  know this!)

# We've made several versions of model neurons.
# We have 3 possible linear filters.
# Toggle between

kernelSize = 60
(linearResp, filter) = getLinear1(stimulus, kernelSize)
# (linearResp, filter) = getLinear2(stimulus, kernelSize)
# (linearResp, filter) = getLinear3(stimulus, kernelSize)

In [ ]:
# The linear response smooths (averages) the stimulus
# over time.  The top panel of the figure shows the first
# second of the stimulus, and the third panel shows the 
# first second of the linear response.

fig, axs = plt.subplots(2, constrained_layout=True, figsize=(6, 5))

axs[0].plot(stimulus[0:1000])
axs[0].set_title('Stimulus', fontsize=16)

axs[1].plot(linearResp[0:1000])
axs[1].set_title('Linear response', fontsize=16)
axs[1].set_xlabel('Time (ms)', fontsize=16)

In [ ]:
# Let's look at the filter (which we usually would not know)

plt.plot(filter, 'o-');
plt.title('Actual filter', fontsize=16)
plt.xlabel('Time (ms)', fontsize=16);

In [ ]:
# We would like to unpack this and see what the linear
# filter is doing
# Choose a random starting point, and examine the stimulus
# starting from the random position and with a length equal
# to the filter length. We'll plot both this stimulus sequence
# and the filter. 

thestart = int(np.random.rand()*100+1)
thelen = len(filter)
thestim = stimulus[thestart:thelen+thestart]
fig, axs = plt.subplots(3, constrained_layout=True, figsize=(6, 8))
axs[0].plot(thestim, 'o-')
axs[0].set_title('Stimulus', fontsize=16)
axs[1].plot(filter, 'o-')
axs[1].set_title('Filter', fontsize=16)

# Plot the point by point multiplication of the filter
# with the stimulus
axs[2].plot(thestim*filter, 'o-')
axs[2].set_title('Stimulus times filter', fontsize=16)
axs[2].set_xlabel('Time (ms)', fontsize=16)

In [ ]:
# The linear filter response to the stimulus, is the sum
# of this point by point multiplication. This results in a
# single number. This is also known as inner product or dot product.

print(np.sum(filter*thestim)) # responses calculated by the inner product
print(linearResp[thelen+thestart-1]) # responses returned by getLinear function (solving a differential equation)

In [ ]:
# What if the input was exactly the linear filter
# or variant of (toggle these)

thestim = filter;
#thestim = -filter;
#thestim = np.flipud(filter);

fig, axs = plt.subplots(3, constrained_layout=True, figsize=(6, 8))
axs[0].plot(thestim)
axs[0].set_title('Stimulus', fontsize=16)
axs[1].plot(filter)
axs[1].set_title('Filter', fontsize=16)
axs[2].plot(thestim*filter)
axs[2].set_title('Stimulus times filter', fontsize=16)
axs[2].set_xlabel('Time (ms)', fontsize=16)


## Non-linear model


In [ ]:
# Under a simple non-linear model, the firing rate of a neuron is
# a single-valued non-linear function of an underlying linear response.
# We can pick any such function we want -- this is done in getNonlinear.
# Here we're applying this non-linear transformation on the linear response of
# our simulated neuron. 

nonlinearResp = getNonlinear1(linearResp)
# Toggle between
# nonlinearResp = getNonlinear2(linearResp)

In [ ]:
# We can plot together the linear and nonlinear response

fig, axs = plt.subplots(2, constrained_layout=True, figsize=(6, 5))
axs[0].plot(linearResp[0:1000])
axs[0].set_title('Linear response', fontsize=16) 
axs[1].plot(nonlinearResp[1:1000], color='r')
axs[1].set_title('Nonlinear function of linear response', fontsize=16)
axs[1].set_xlabel('Time (ms)', fontsize=16)

In [ ]:
# To understand this better, we try just few values
# Try changing theval: Is there a pattern?

theval = 220
print(linearResp[theval])
print(nonlinearResp[theval])

In [ ]:
# Plot nonlinear function in the interval [-.3:.05:.3]
# Toggle between

modelNonlin = getNonlinear1(np.arange(-.3,.35,.05))
# modelNonlin = getNonlinear2(np.arange(-.3,.35,.05))

plt.plot(np.arange(-.3,.35,.05), modelNonlin, '-o');

plt.xlabel('Linear response', fontsize=16)
plt.ylabel('Nonlinear response', fontsize=16)

## Spike train

In [ ]:
# We can use this non-linear response to simulate a 
# Poisson-ish spike train... as per last class!

xr = np.random.rand(len(nonlinearResp))
neuralResponse = nonlinearResp > .05*xr
spikeCounts = neuralResponse

In [ ]:
# So far, we constructed a model neuron and its response to experimental
# stimuli. We first constructed random Gaussian stimulus, we linearly 
# filtered it, put this linearly filtered signal through a non-linear 
# function to calculate an underlying firing rate of the cell, and used 
# this to simulate spikes coming out of the cell.  
# Here's the first second of each of these:


fig, axs = plt.subplots(3, constrained_layout=True, figsize=(6, 8))
axs[0].plot(linearResp[0:1000])
axs[0].set_title('Linear response', fontsize=16) 
axs[1].plot(nonlinearResp[1:1000], color='r')
axs[1].set_title('Nonlinear function of linear response', fontsize=16)
axs[2].stem(neuralResponse[1:1000], basefmt=" ")
axs[2].set_title('# of Spikes (1 ms bins)', fontsize=16)
axs[2].set_xlabel('Time (ms)', fontsize=16)

## Things to do in class:

1. Try changing the linear function (choosing between getLinear1,
getLinear2, getLinear3; see toggle comment). 

2. Try changing the nonlinear function (choose between getNonlinear1,
getNonlinear2.m). Change it in the two places in the tutorial.
What is the difference between getNonlinear1 and getNonlinear2?
